# fucntion calling

![function-calling flow chart](./imgs/flow-chart.svg)

| Step | Executor   | Key Action                     | Data Format Example                                           |
| :--- | :--------- | :----------------------------- | :------------------------------------------------------------ |
| 1    | Application | Send user input + function definitions | `{"messages":[...], "functions":[...]}`                      |
| 2    | Large Model | Decide response method         | Generate `tool_calls` or respond directly                     |
| 3    | Large Model | Return function call           | `{"tool_calls": [{"name": "func", "arguments": {...}}]}`     |
| 4    | Application | Execute local function         | Parse arguments and invoke `func(**args)`                    |
| 5    | Application | Submit execution result        | `{"tool_call_id": "...", "role": "tool", "content": "result"}` |

Chinese vesion:
| 步骤 | 执行方 | 关键动作              | 数据格式示例                                                 |
| :--- | :----- | :-------------------- | :----------------------------------------------------------- |
| 1    | 应用   | 发送用户输入+函数定义 | `{"messages":[...], "functions":[...]}`                      |
| 2    | 大模型 | 决策响应方式          | 生成`tool_calls`或 直接回答                                   |
| 3    | 大模型 | 返回函数调用          | `{"tool_calls": [{"name": "func", "arguments": {...}}]}`     |
| 4    | 应用   | 执行本地函数          | 解析参数并调用`func(**args)`                                 |
| 5    | 应用   | 提交执行结果          | `{"tool_call_id": "...", "role": "tool", "content": "result"}` |

step 5 is optional

addional flowchart to understand it better

 <img src="./imgs/flow-chart2.svg" alt="Flow Chart" width="400" />

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
def get_weather(location):
    return "24℃"
available_functions = {"get_weather": get_weather}

In [ ]:
from openai import OpenAI
import os

def send_messages(messages):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        tools=tools
    )
    return response.choices[0].message

client = OpenAI()

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather of an location, the user shoud supply a location first",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"]
            },
        }
    },
]

In [ ]:
messages = [{"role": "user", "content": "How's the weather in Hangzhou?"}]
message = send_messages(messages)
messages.append(message)
print(f"User>\t {messages[0]['content']}")
print(f"model response with tool_to_call\n: {message}")

tool = message.tool_calls[0]

User>	 How's the weather in Hangzhou?
model response with tool_to_call: ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_47c3cbc2-1a7d-444f-9f86-a367822ce9ae', function=Function(arguments='{"location":"Hangzhou"}', name='get_weather'), type='function', index=0)])


观察上面打印的LLM结果，其中`content`为空，`tool_calls`不空，表示LLM告诉我们下一步需要调用对应的tools。相反，如果LLM返回的是直接结果，则content不空，tool_calls为空。

In [ ]:
print(tool)
print(tool.function.name)
print(tool.function.arguments)
print(type(tool.function.arguments))

ChatCompletionMessageToolCall(id='call_0_47c3cbc2-1a7d-444f-9f86-a367822ce9ae', function=Function(arguments='{"location":"Hangzhou"}', name='get_weather'), type='function', index=0)
get_weather
{"location":"Hangzhou"}
<class 'str'>


In [ ]:
import json
function_name = available_functions[tool.function.name]
function_args = json.loads(tool.function.arguments)
function_response = function_name(**function_args) #call the function
function_response

'24℃'

为什么要传 `tool_call_id`： 可能会涉及到多个tool的调用，为了让LLM匹配哪个tool的结果是什么。

In [ ]:
 #append the function response into the messages, which will be sent to the model to let model know the result of the tool
messages.append({"role": "tool", "tool_call_id": tool.id, "content": function_response})
message = send_messages(messages)
print(f"Model>\t {message.content}") 
messages.append(message)

Model>	 The current weather in Hangzhou is 24°C. It's a pleasant temperature!


In [ ]:
messages

[{'role': 'user', 'content': "How's the weather in Hangzhou?"},
 ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_47c3cbc2-1a7d-444f-9f86-a367822ce9ae', function=Function(arguments='{"location":"Hangzhou"}', name='get_weather'), type='function', index=0)]),
 {'role': 'tool',
  'tool_call_id': 'call_0_47c3cbc2-1a7d-444f-9f86-a367822ce9ae',
  'content': '24℃'},
 ChatCompletionMessage(content="The current weather in Hangzhou is 24°C. It's a pleasant temperature!", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)]